
<div style="background-color:#000047; padding: 30px; border-radius: 10px; color: white; text-align: center;">
    <img src='Figures/alinco.png' style="height: 100px; margin-bottom: 10px;"/>
    <h1> Arboles de Decision y Bosques Aleatorios</h1>
</div>


## Arboles de Desición


Los árboles de decisión, también conocidos como modelos de árbol de clasificación y regresión (CART), son métodos basados en árboles para el aprendizaje automático supervisado. Los árboles de clasificación y de regresión simples son fáciles de usar e interpretar, pero no son competitivos con los mejores métodos de aprendizaje automático. Sin embargo, forman la base para el conjunto de modelos de ensamblaje como “bagged trees”, “random forest” y “boosted trees”, que aunque son menos interpretables, son muy precisos.

Los modelos CART se puede definir en dos tipos de problemas

**Árboles de clasificación:** la variable resultado es categórica y el métodos se utiliza para identificar la “clase” dentro de la cual es más probable que caiga nuestra variable resultado. Un ejemplo de un problema de tipo clasificación sería determinar quién se suscribirá o no a una plataforma digital; o quién se graduará o no de la escuela secundaria; o si una persona tiene cáncer o no.

**Árboles de regressión:** la variable resultado es continua y el métodos se utiliza para predecir su valor. Un ejemplo de un problema de tipo regresión sería predecir los precios de venta de una casa residencial o el nivel de colesterol de una persona.


Un árbol de decisión es una secuencia de operadores relacionales organizados como árbol donde:

- Los atributos de un dato son evaluados desde la raiz hasta las hojas
- Los nodos hoja (terminales) están asociados a una clase
- Los nodos no-hoja están asociados a un operador lógico que divide los datos en dos o más conjuntos
- El operador lógico o *split* se aplica sobre un atributo (feature) de los datos

El siguiente diagrama ejemplifica el funcionamiento del árbol de decisión sobre un dataset con dos etiquetas y dos atributos (X y Z). 

<img src="Figures/tree.png" width="600">

- La figura izquierda muestra un árbol de decisión binario con 5 nodos: 3 nodos hoja y 2 nodos de decisión.
- La figura derecha muestra la partición que produce el árbol de decisión en el espacio de los datos. 
- Las separaciones o *splits* son siempre perpendiculares a los ejes de los datos (atributos).


Entrenar el árbol de decisión es el proceso de escoger los atributos, operadores y umbrales de separación en los nodos de decisión.


La función de costo más común para los árboles de regresión es la suma de los residuos al cuadrado,

$$RSS = \sum_{k=1}^{K}\sum_{i \ in A_k} (y_i - \hat{y}_{A_k})^2$$

Para árboles de clasificación, es el índice de Gini,

$$ G=\sum_{c=1}^C \hat{p}_{kc} (1-\hat{p}_{kc})$$

y la entropía (información estadística)

$$ E= - \sum_{c=1}^C \hat{p}_{kc} log(\hat{p}_{kc})$$

dónde $\hat{p}_{kc}$ es la proporción de observaciones de entrenamiento en el nodo $k$ que son de clase $c$. Un nodo completamente puro en un árbol binario tendría $\hat{p} \in \{0,1\}$ y $G=E=0$.  Un nodo completamente impuro en un árbol binario tendría $\hat{p}=0.5$ y $G=0.5^2*2 = 0.25$, y $D=- (0.5 \cdot log(0.5))\cdot 2 = 0.69$

La ganacia de información para un nodo que separa un conjunto de datos $D$ en dos $D_{izq}$ y $D_{der}$ es

$$
G(D; D_{izq}, D_{der}) = H(D) - \frac{|D_{izq}|}{|D|} H(D_{izq}) - \frac{|D_{der}|}{|D|} H(D_{der})
$$

donde $|A|$ es la cardinalidad del subconjunto $A$ y 

$$
H(A) = - \sum_{y \in \mathcal{Y}} p(y|A) \log p(y|A)
$$

es la entropía del subconjunto $A$. En la expresión anterior $p(y|A)$ es la frecuencia relativa de los ejemplos de clase $y$ dentro de $A$.

**La entropía mide la “pureza” del subconjunto en términos de sus clases. El subconjunto más puro es aquel donde todos los elementos son de la misma clase. El nodo más impuro es aquel en donde hay igual cantidad de elementos de cada clase (uniforme).**

## Algoritmo

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

Sea el siguiente arreglo las etiquetas de un subconjunto de 12 ejemplos

In [ ]:
labels = np.array([1,0,0,1,1,1,0,1,0,1,1,1])
labels

Asumiendo que el problema sólo tiene dos clases las frecuencias relativas son

In [ ]:
a, counts= np.unique(labels, return_counts=True)
counts/len(labels)


Y la entropía del conjunto sería:

In [ ]:
def entropy(subset_labels):
    unique, counts= np.unique(subset_labels, return_counts=True)
    frecuencias = counts/len(subset_labels)
    entropia = -np.sum(frecuencias*np.log2(frecuencias +1e-16))
    return entropia


In [ ]:
entropy(labels)

La entropía es máxima si hay igual cantidad de ejemplos de ambas clases (mínima pureza)

In [ ]:
B = np.array([1,1,1,0,0,0])
entropy(B)

y mínima si todos los ejemplos son de una clase  (máxima pureza)

In [ ]:
C = np.array([1,1,1,1,1,1])
entropy(C)

**Extensión a más de dos clases**

Si un nodo separa el conjunto en $k$ subconjuntos la regla es


- En cada nodo se escoge el atributo que maximiza la ganancia de información.

Consideremos el siguiente problema:

In [ ]:
data = {'tiempo' : ['soleado', 'soleado', 'soleado', 'lluvioso', 'lluvioso'],
        'humedad' : ['baja','baja', 'alta','alta','alta'],
        'temperatura': ['templado','caluroso', 'caluroso', 'templado', 'frio']}

nodo = pd.DataFrame(data)
nodo

donde queremos obtener un árbol de decisión que prediga el tiempo en función de la humedad y de la temperatura.

Para decidir cual variable debe ir en el primer nodo comparamos sus ganancias de información

In [ ]:
def info_gain(subset, feature):
    subset_labels = subset['tiempo'].values
    entropy_root = entropy(subset_labels)
    entropy_nodes = []
    for unique_label in subset[feature].unique():
        split = subset.loc[subset[feature] == unique_label]
        split_labels = split['tiempo'].values
        entropy_nodes.append(entropy(split_labels)*len(split_labels)/len(subset_labels))
    return entropy_root - sum(entropy_nodes)
    

In [ ]:
for feature in ['humedad','temperatura']:
    print(f'La ganancia de información para {feature} es:{info_gain(nodo, feature)} ')

Temperatura tiene mayor ganancia que humedad, por lo tanto el primer nodo separador utiliza temperatura.

Si separamos por temperatura tenemos:

In [ ]:
nodo.loc[nodo['temperatura']=='frio']

En el caso `frio` se produce un nodo con un sólo ejemplo. El algoritmo no seguirá dividiendo.

In [ ]:
nodo.loc[nodo['temperatura']=='caluroso']

En el caso `caluroso` se produce un nodo con "puro". El algoritmo no seguirá dividiendo.

In [ ]:
nodo.loc[nodo['temperatura']=='templado']

En el caso `templado` el nodo no es puro, debemos nuevamente escoger un atributo para separar:

In [ ]:
nodo_next = nodo.loc[nodo['temperatura']=='templado']
for feature in ['temperatura', 'humedad']:
    print(f'La ganancia de información para {feature} es:{info_gain(nodo_next, feature)} ')

Por lo tanto se escoge humedad, lo cual produce dos nodos puros (con un sólo ejemplo)

In [ ]:
nodo_next.loc[nodo_next['humedad']=='baja']

In [ ]:
nodo_next.loc[nodo_next['humedad']=='alta']

El algoritmo sigue separando el dataset de forma recursiva hasta que todos los nodos sean puros o hasta que se supere una profundidad máxima previamente designada.


## Creación de la clase de Arboles de Desicion


In [ ]:
import numpy as np
from collections import Counter

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        
    def is_leaf_node(self):
        return self.value is not None

class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # criterio de paro
        if (depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        # mejor split
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        # Creación de los nodos hijos
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feature, best_thresh, left, right)


    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                # Ganancio de información
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold


    def _information_gain(self, y, X_column, threshold):
        # pentropia del padre
        parent_entropy = self._entropy(y)

        # creando un nodo hijo
        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        
        # calculando el peso medio de la entropia de los nodos hijos
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        # Ganancia de información
        information_gain = parent_entropy - child_entropy
        return information_gain

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log(p) for p in ps if p>0])


    def _most_common_label(self, y):
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)
    
    

## Ejemplo 1

In [ ]:
from sklearn import datasets
data = datasets.load_breast_cancer()


In [ ]:
X, y= data.data, data.target

## Ejemplo 2:

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
y = iris.target
target_name = iris.

In [ ]:
df_iris = pd.DataFrame(data=X, columns = iris.feature_names)
df_iris['especie'] = y

In [ ]:
df_iris['especie'].unique()

In [ ]:
df_iris.head(3)

## Implementación en scikit-learn

El módulo [`tree`](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.tree) de scikit-learn tiene implementaciones de árboles de decisión para problemas de clasificación y regresión. Nos enfocaremos en la primera.

Los principales argumentos de [`DecisionTreeClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier) son:

- `criterion`: El criterio que se utiliza para escoger los *splits*, las opciones son `'gini'` y `'entropy'`
- `max_depth`: Límite para la profundidad máxima del árbol
- `min_samples_split`: El número mínimo de ejemplos en un nodo para realizar un *split*
- `min_samples_leaf`: El número mínimo de ejemplos que pueden estar en un nodo hoja
- `min_impurity_decrease`: La disminución de pureza mínima en un nodo para realizar un *split*
- `class_weight`: Permite asignar ponderación a las clases, es de utilidad si se tienen clases medianamente desbalanceadas
- `max_features`: El número máximo de atributos a considerar en cada *split*


Si se utilizan los argumentos (hiperparámetros) por defecto el árbol crecera hasta que sus nodos sean todos puros. Esto en general produce árboles de gran profundidad (muy capaces de sobreajustarse). 

Se puede limitar el tamaño de un árbol aumentando `min_samples_leaf` y/o `min_samples_split`, o disminuyendo `max_depth`.



Los principales métodos son:

- `predict(X)`: Retorna la clase predicha
- `predict_proba(X)`: Retorna las probabilidades de pertenecer a cada una de las clases
- `score(X,y)`: Retorna el *accuracy* de clasificación
- `get_params()`: Retorna los nombres de los parámetros

Además tiene algunos métodos no compartidos con otros estimadores como

- `get_depth()`: Retorna la profunidad del árbol aprendido
- `get_n_leaves()`: Retorna la cantidad de nodos hoja del árbol aprendida
- `apply(X)`: Retorna el índice de la hoja que predice cada ejemplo



Podemos utilizar la función [`plot_tree`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.plot_tree.html#sklearn.tree.plot_tree) para obtener una visualización del árbol de decisión. En cada nodo se muestra:

- El atributo y umbral seleccionados.
- El valor del criterio (índice de gini).
- La cantidad de ejemplos que entraron al nodo.
- La cantidad de ejemplos que entraron al nodo separados por clase (en este caso tres).


In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

In [ ]:
#Creación del modelo
model_iris = DecisionTreeClassifier(criterion='entropy')
model_iris.fit(X_train_df, y_train_df)

## Bosques Aleatorios (Random Forest)

**Random Forest** es un modelo de conjunto (ensemble) que combina múltiples árboles de decisión para mejorar la precisión y reducir el sobreajuste.

<b>Algoritmo</b>
<ul>
<li>Se construyen muchos árboles de decisión independientes.</li>
<li>Cada árbol se entrena con una muestra aleatoria de los datos (bootstrap) y una selección aleatoria de características en cada división (feature bagging).</li>
<li>Para clasificación, cada árbol vota por una clase y la clase final es la más votada (mayoría).</li>
<li>Para regresión, se promedian las predicciones de todos los árboles.</li>
</ul>

<b>Proceso de construcción:</b>
<ol>
<li>Se generan <i>N</i> subconjuntos de datos mediante muestreo con reemplazo (bootstrap).</li>
<li>Para cada subconjunto, se construye un árbol de decisión usando solo un subconjunto aleatorio de características en cada nodo.</li>
<li>Las predicciones de todos los árboles se combinan (votación o promedio).</li>
</ol>

<b>Ejemplo de aplicación:</b> Random Forest es ampliamente usado en tareas de clasificación (diagnóstico médico, detección de fraude, etc.) y regresión (predicción de precios, estimación de demanda, etc.).

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from HyAIA import HyAIA as hya

In [ ]:
df = pd.read_csv("Data/diabetes.csv")

In [ ]:
df_diabetes = hya(df)

#### Preprocesamiento

Tratamiento de Outliers

In [ ]:
#Outliers


In [ ]:
# Eliminando outliers (forma básica)


In [ ]:
#Datos imbalanceados


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, recall_score, precision_score,f1_score


In [ ]:
#Train test split


In [ ]:
#Función de evaluación


#### Modelo de arbol de desición

In [ ]:
#Modelo


#### Modelo de Arboles Aleatorios

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

<b>Resumen:</b>
<ul>
<li>Reduce el riesgo de sobreajuste que tienen los árboles individuales.</li>
<li>Generaliza mejor en datos nuevos.</li>
<li>Puede manejar grandes cantidades de variables y datos.</li>
<li>Permite estimar la importancia de cada variable.</li>
</ul>

<b>algunas desventajas:</b>
<ul>
<li>Menos interpretable que un solo árbol de decisión.</li>
<li>Puede ser más lento en entrenamiento y predicción si el número de árboles es grande.</li>
</ul>
